In [1]:
import json as js
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
# script to download all the nomination data from prorepublica
# create a list. Use list comprehension

# Find the roll calls for nominees. 

import requests
header={"X-API-Key": "nPbsYphjkf8RIEQ0EBGHt4YsISgl2bxX2MrmrIOq"}


In [3]:
# Get Congress Members
url_members = 'https://api.propublica.org/congress/v1/115/house/members.json'
mresp = requests.get(url_members,headers=header)
print mresp.status_code
d = mresp.json()
members = d['results'][0]['members']
df = json_normalize(members)
df.to_csv('./data/115members.csv', encoding='utf-8', index=False)

200


In [4]:
# Get Senate Members
url_senate = 'https://api.propublica.org/congress/v1/115/senate/members.json'
nresp = requests.get(url_senate,headers=header)
print nresp.status_code
s = nresp.json()
senate = s['results'][0]['members']
df = json_normalize(senate)
df.to_csv('./data/115senators.csv', encoding='utf-8', index=False)

200


In [5]:
# Get Nomination Data

url_nom = 'https://api.propublica.org/congress/v1/115/nominations.json'
nresp = requests.get(url_nom,headers=header)
print nresp.status_code
nom = nresp.json()
nom_temp=nom['results'][0]['votes']
df = json_normalize(nom_temp)
roll_call_list = df.roll_call.unique()
df.to_csv('./data/115nom.csv', encoding='utf-8', index=False)

200


In [12]:
# Get detail Nomination Votes
# roll_call_list = 61
url_detail_nom_base = 'https://api.propublica.org/congress/v1/115/senate/sessions/1/votes/'
for i in roll_call_list:
    url_detail_nom = url_detail_nom_base + str(i)+'.json'
    detail_nresp = requests.get(url_detail_nom,headers=header)
    print detail_nresp.status_code
    detail_nom = detail_nresp.json()
    temp_nom = detail_nom['results']['votes']['vote']
    df = json_normalize(temp_nom, 'positions', ['roll_call'])
    file_name = './data/nom'+ str(i) + '.csv'
    df.to_csv(file_name, encoding='utf-8', index=False)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [13]:
# DATA WRANGLING

# combine all individual roll call vote files into 1
fout=open("./data/nom.csv","a")
# first file:
for line in open("./data/nom" + str(roll_call_list[0])+ ".csv"):
    fout.write(line)
# now the rest:    
for i in roll_call_list[1:]:
    f = open("./data/nom"+str(i)+".csv")
    f.next() # skip the header
    for line in f:
         fout.write(line)
    f.close() # not really needed
fout.close()


In [14]:
# Make a clean dataset
temp_df = pd.read_csv('./data/nom.csv')
temp_df.drop('dw_nominate', axis=1, inplace=True)
#temp_df.drop(temp_df.columns[0], axis=1, inplace=True)
temp_df.head()
temp_df.to_csv('./data/cnom.csv', index=False)

In [10]:
# Clean data for Nomination Reference File and Member Reference File

temp_df = pd.read_csv('./data/115members.csv')
temp_df.drop(['dw_nominate','api_uri', 'facebook_account','google_entity_id', 'ideal_point','rss_url'], axis=1, inplace=True)
temp_df.rename(columns={'id': 'member_id'}, inplace=True)
temp_df.head()
temp_df.to_csv('./data/c115members.csv', index=False)

In [11]:
# Clean data for Senator Reference File

temp_df = pd.read_csv('./data/115senators.csv')
temp_df.drop(['dw_nominate','api_uri', 'facebook_account','google_entity_id', 'ideal_point','rss_url'], axis=1, inplace=True)
temp_df.rename(columns={'id': 'member_id'}, inplace=True)
temp_df.head()
temp_df.to_csv('./data/c115senators.csv', index=False)

In [ ]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [ ]:
nom = pd.read_csv('./data/cnom.csv')
mem115 = pd.read_csv('./data/c115members.csv')

In [ ]:
#nom['member_id']
mem115['member_id']